## Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Activation Functions

In [2]:
class Activations:
    @staticmethod
    def step(x):
        if x < 0:
            return 0
        return 1
    
    @staticmethod
    def sigmoid(x):
        return 1 / (1+np.exp(-x))

    @staticmethod
    def tanh(x):
        return (2 / (1+np.exp(-2*x))) - 1

    @staticmethod
    def relu(x):
        print("x relu: ", x)
        return np.max(0, x)

    @staticmethod
    def leaky_relu(x):
        if x>=0:
            return x
        return 0.3*x
    
    @staticmethod
    def elu(x, alpha=1):
        if x>=0:
            return x
        return alpha*(np.exp(x)-1)
    
    @staticmethod
    def binary(x):
        if x>= 0.5:
            return 1
        return 0

## Activation Function derivates to compute gradients

In [3]:
class ActivationDerivatives:
    @staticmethod
    def step_deriv(x):
        return 0

    @staticmethod
    def sigmoid_deriv(x):
        sigmoid = Activations.sigmoid(x)
        return sigmoid*(1-sigmoid)

    @staticmethod
    def tanh_deriv(x):
        return 1 - np.sqrt(Activations.tanh(x))

    @staticmethod
    def relu_deriv(x):
        if x<0:
            return 0
        return 1

    @staticmethod
    def leaky_relu_deriv(x):
        if x>=0:
            return 1
        return 0.3

    @staticmethod
    def elu_deriv(x, alpha=1):
        if x<0:
            return Activations.elu(x)+alpha
        return 1

# Losses

In [4]:
class Loss:
    @staticmethod
    # Regression Mean Squared Error
    def mse(targets, predictions):
        differences_squared = (predictions-targets)**2
        return np.sum(differences_squared)
    
    @staticmethod
    # Binary Cross Entropy
    def binary_cross_entropy(y_target, prob):
        return -(y_target*np.log(prob) + (1-y_target)*np.log(1-prob))
    
    @staticmethod
    def mse_prime(y_target, prediction):
        return prediction-y_target

# Build ANN

## Build Layers

In [5]:
class Dense():
    def __init__(self, units, activation, input_dim = None):
        available_activations = ['step', 'sigmoid', 'tanh', 'relu', 'leaky_relu', 'elu']
        activations = [Activations.step, Activations.sigmoid, Activations.tanh, Activations.relu, Activations.leaky_relu, Activations.elu]
        activation_derivs = [ActivationDerivatives.step_deriv, ActivationDerivatives.sigmoid_deriv, ActivationDerivatives.tanh_deriv, ActivationDerivatives.relu_deriv, ActivationDerivatives.leaky_relu_deriv, ActivationDerivatives.elu_deriv]
        index = available_activations.index(activation)
        if not index:
            print("Error. Activation not found, list of available activations are: ", available_activations)
            return
        self.units = units
        self.input_dim = input_dim
        self.activation = activations[index]
        self.activation_deriv = activation_derivs[index]
        
        self.input = []
        self.activations = []
        
    def compile_layer(self, input_dim=None):
        if (self.input_dim != None):
            input_dim = self.input_dim
        self.weights = np.random.rand(input_dim, self.units) # initialize weights
        self.b = np.random.rand()
        return self.units
        
    def propagate(self, x):
        z = np.dot(x, self.weights) + self.b
        self.input = z
        self.activations = self.activation(z)
        return self.activations
    
    def backpropagate(self, local_gradient, learning_rate):
        errors = np.multiply(local_gradient, self.activations)
        layer_gradient = np.array([self.activation_deriv(a) for a in self.activations])*errors
        for gradient in layer_gradient:
            self.b -= gradient
        for index in range(self.weights.shape[0]):
            self.weights[index] -= (learning_rate * layer_gradient[index] * self.input[index])
        return layer_gradient

In [6]:
class Model():
    def __init__(self, lr=0.01):
        self.layers = []
        self.loss = Loss.mse_prime
        self.learning_rate = lr
        
    def compile_layers(self):
        input_dim = self.layers[0].compile_layer()
        for i in range(1, len(self.layers)):
            input_dim = self.layers[i].compile_layer(input_dim)
    
    def fit(self, x, y, epochs=20, batch_size=4):
        dataset_size=x.shape[0]
        if batch_size>dataset_size:
            print("Error. Batch size cannot be greater than dataset size")
            return
        for epoch in range(epochs):
            # assume batch size is 1 for now
            batch_start = 0
            batch_end = batch_size
            while batch_end < dataset_size:
                if (batch_end > dataset_size):
                    batch_end = dataset_size
                x_sample = x[batch_start:batch_end]
                y_batch = y[batch_start:batch_end]
                predictions = self.propagate(x_sample)
                self.backpropagate(y_batch, predictions)
                batch_start+=batch_size
                batch_end+=batch_size
    
    def predict(self, x):
        predictions = []
        for x_sample in x:
            predictions.append(self.propagate(x_sample))
        return predictions
    
    def add(self, layer):
        self.layers.append(layer)
    
    def propagate(self, x_batch):
        for i in range(len(self.layers)):
            x_batch = np.array(self.layers[i].propagate(x_batch))
        prediction = x_batch
        return prediction
    
    def backpropagate(self, y_target, prediction_values):
        output_layer = self.layers[-1]
        errors = self.loss(y_target, prediction_values)
        print("loss: ", np.mean(errors))
        local_gradient = np.array([output_layer.activation_deriv(a) for a in output_layer.activations])*errors
        output_layer.weights -= self.learning_rate*np.dot(output_layer.input.T, local_gradient)
        for gradient in local_gradient:
            output_layer.b -= gradient
        for i in range(len(self.layers)-2, -1, -1):
            self.layers[i].backpropagate(local_gradient, self.learning_rate)
            
    
    def summary(self):
        pass


# Set up Data 

In [7]:
breast_cancer_data = pd.read_csv('Data/breast_cancer_diagnosis_data.csv')
breast_cancer_data.drop(['id'], axis=1, inplace=True)
breast_cancer_data.dropna(axis='columns', inplace=True)
breast_cancer_data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [8]:
np.unique(breast_cancer_data['diagnosis']) #Malignant or Benign
def diagnosis_to_binary(diagnosis):
    if diagnosis=='B':
        return 0
    elif diagnosis=='M':
        return 1
    
breast_cancer_data['diagnosis'] = breast_cancer_data['diagnosis'].apply(diagnosis_to_binary)

## Feature Selection

In [9]:
def standard_units(arr):
    return (arr-np.mean(arr))/np.std(arr)
                   
def correlation(a, b):
    return np.mean(standard_units(a)*standard_units(b))

In [10]:
def correlations(df, y_feature):
    feature_correlations = {}
    y = df[y_feature].values
    df_copy = df.drop([y_feature], axis = 1)
    print("Explore Correlation between features and " + str(y_feature))
    for column in df_copy:
        corr = correlation(df[column].values, y)
        feature_correlations[column] = corr
        print(str(column)+" Correlation: " + str(corr))
    return feature_correlations

In [11]:
feature_correlations = correlations(breast_cancer_data, 'diagnosis')

Explore Correlation between features and diagnosis
radius_mean Correlation: 0.7300285113754565
texture_mean Correlation: 0.4151852998452045
perimeter_mean Correlation: 0.7426355297258332
area_mean Correlation: 0.7089838365853901
smoothness_mean Correlation: 0.3585599650859321
compactness_mean Correlation: 0.5965336775082535
concavity_mean Correlation: 0.6963597071719059
concave points_mean Correlation: 0.7766138400204355
symmetry_mean Correlation: 0.33049855426254715
fractal_dimension_mean Correlation: -0.012837602698432387
radius_se Correlation: 0.5671338208247177
texture_se Correlation: -0.008303332973877421
perimeter_se Correlation: 0.5561407034314833
area_se Correlation: 0.5482359402780244
smoothness_se Correlation: -0.06701601057948733
compactness_se Correlation: 0.2929992442488584
concavity_se Correlation: 0.25372976598083036
concave points_se Correlation: 0.40804233271650475
symmetry_se Correlation: -0.006521755870647961
fractal_dimension_se Correlation: 0.07797241739025616
radi

In [12]:
def pick_correlations(feature_correlations, threshold=0.8):
    correlations_chosen = []
    for feature in feature_correlations:
        if feature_correlations[feature] >= threshold:
            correlations_chosen.append(feature)
    return correlations_chosen

In [13]:
# choose features with correlation higher than 0.75
columns_to_keep = pick_correlations(feature_correlations, threshold=0.73)

In [14]:
breast_cancer_data_diagnosis = breast_cancer_data[['diagnosis']]
breast_cancer_data_features = breast_cancer_data[columns_to_keep]
breast_cancer_data_features.head()

,radius_mean,perimeter_mean,concave points_mean,radius_worst,perimeter_worst,area_worst,concave points_worst
0,17.99,122.80,0.14710,25.38,184.60,2019.0,0.2654
1,20.57,132.90,0.07017,24.99,158.80,1956.0,0.1860
2,19.69,130.00,0.12790,23.57,152.50,1709.0,0.2430
3,11.42,77.58,0.10520,14.91,98.87,567.7,0.2575
4,20.29,135.10,0.10430,22.54,152.20,1575.0,0.1625


## Split Tran and Test

In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(breast_cancer_data_features, breast_cancer_data_diagnosis, test_size=0.33, random_state=42)

In [16]:
from sklearn.preprocessing import StandardScaler

# make sure they are in numpy form
x_train = np.array(x_train.values, dtype=np.float32)
x_test = np.array(x_test.values, dtype=np.float32)
y_train = np.array(y_train.values, dtype=np.uint8)
y_test = np.array(y_test.values, dtype=np.uint8)

# scale data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

print("x train samples: ", x_train[:5])
print("x test samples: ", x_test[:5])
print("y train samples: ", y_train[:5])
print("y test samples: ", y_test[:5])

x train samples:  [[ 0.35581988  0.4130312   1.5938771   0.5110286   0.52179366  0.37393498
   1.0565228 ]
 [-0.37326664 -0.39470002 -0.7896234  -0.39223224 -0.473485   -0.4140696
  -0.8966741 ]
 [ 1.4033579   1.3479992   1.0764874   2.0233212   1.8669261   2.1755323
   1.4635353 ]
 [-0.47941723 -0.48331875 -0.6581813  -0.46218845 -0.4379925  -0.48057154
  -0.13676336]
 [-0.48221073 -0.48413163 -0.46704042 -0.585641   -0.5778744  -0.57730156
  -0.5846312 ]]
x test samples:  [[-0.47941723 -0.45730227 -0.27799425 -0.27495223 -0.34165588 -0.3586225
  -0.19534856]
 [ 1.3279356   1.2707627   0.8033918   1.7599561   1.7416584   1.6936095
   0.9979379 ]
 [ 0.35581988  0.38051045  0.8390016   0.6077329   0.51881117  0.46721038
   0.5739667 ]
 [-0.49897146 -0.44185477 -0.5455915  -0.7049784  -0.5337325  -0.63862157
  -0.6239449 ]
 [-0.73920673 -0.7190931  -0.5992681  -0.83254594 -0.8567432  -0.73880625
  -0.69363046]]
y train samples:  [[1]
 [0]
 [1]
 [0]
 [0]]
y test samples:  [[0]
 [1]
 [1]
 

# Train Model

In [17]:
# create model
model = Model(lr=0.008)

# add Hidden Layer Layer
hl1 = Dense(units=15, activation='sigmoid', input_dim=len(x_train[0]))
model.add(hl1)

# add Output Layer
output_layer = Dense(units=1, activation='sigmoid')
model.add(output_layer)

model.compile_layers()

In [18]:
model.layers[0].weights

array([[0.55265752, 0.08597444, 0.97116372, 0.02537418, 0.47764333,
        0.08152132, 0.04942744, 0.34541515, 0.89264017, 0.63157816,
        0.24526856, 0.75078993, 0.95711801, 0.29245248, 0.60377551],
       [0.40742637, 0.4623494 , 0.95435794, 0.16714366, 0.14198611,
        0.81768297, 0.27639106, 0.39146769, 0.16630764, 0.4495227 ,
        0.89581184, 0.45834406, 0.79392086, 0.68615689, 0.57293104],
       [0.01530229, 0.63182555, 0.02954806, 0.94485143, 0.95248882,
        0.83616807, 0.52936851, 0.68291733, 0.80616517, 0.89106626,
        0.20844643, 0.53636482, 0.32042409, 0.42393111, 0.42194707],
       [0.09635632, 0.88691829, 0.31574932, 0.51106471, 0.70319287,
        0.60564979, 0.41540521, 0.79928236, 0.55270272, 0.80591693,
        0.34359588, 0.89365707, 0.41794158, 0.46104534, 0.9168771 ],
       [0.91166864, 0.62544276, 0.04714605, 0.14712283, 0.3338299 ,
        0.97827052, 0.85935528, 0.25156122, 0.27974481, 0.42861186,
        0.37325665, 0.78010948, 0.8537368 , 

In [19]:
model.layers[1].weights

array([[0.93528617],
       [0.34483874],
       [0.81261291],
       [0.72919684],
       [0.60832099],
       [0.48575556],
       [0.42326382],
       [0.82876587],
       [0.63998127],
       [0.13354023],
       [0.65433301],
       [0.05293887],
       [0.29941859],
       [0.04039963],
       [0.86075402]])

In [20]:
model.fit(x_train, y_train, epochs=80, batch_size=8)

loss:  0.5382399098899169
loss:  0.5522435869215307
loss:  0.2757445145860089
loss:  0.23525863705461808
loss:  0.20044595570703427
loss:  0.21180186078375413
loss:  0.18812564068867
loss:  0.11295023823764785
loss:  0.12762577537773823
loss:  0.12316855852839852
loss:  0.15863234574029564
loss:  -0.050702074506376565
loss:  0.05858924895237789
loss:  0.06839795368356029
loss:  -0.12957617820808442
loss:  -0.07821884702132956
loss:  0.10117257197453795
loss:  0.00786600386072207
loss:  0.08890207827790056
loss:  0.046431340907757
loss:  0.02291033227254015
loss:  0.08522608983077665
loss:  0.010119327829482487
loss:  -0.01527100465695794
loss:  0.07344169781301833
loss:  0.10264270153036922
loss:  -0.09089467384006819
loss:  0.04013408001509802
loss:  0.08044298572945727
loss:  -0.15577269827477466
loss:  -0.025546484086600253
loss:  -0.151698570919221
loss:  0.03385878858924078
loss:  -0.0003994655530039342
loss:  -0.008255542615663757
loss:  -0.052316088219172996
loss:  -0.1637904834

loss:  -0.016374809087036476
loss:  0.06625083963182407
loss:  0.09971608607441179
loss:  -0.09279504770430824
loss:  0.03923475009568043
loss:  0.061613284487102064
loss:  -0.15190535954140655
loss:  -0.022354131343325902
loss:  -0.15977089235551004
loss:  0.03422033217820537
loss:  -0.005193844219973597
loss:  -0.012278675062902178
loss:  -0.057387419229404185
loss:  -0.18117798732033258
loss:  0.018492646576385944
loss:  0.052378749187806474
loss:  -0.008282450618365477
loss:  0.0381784423346459
loss:  0.07374646191269664
loss:  -0.021109720043349912
loss:  0.1275351399248197
loss:  0.002952310472892968
loss:  0.10183120185884967
loss:  -0.03193408213861759
loss:  -0.010248353778264767
loss:  0.024983838320187496
loss:  -0.07798456194505325
loss:  -0.05368697902543836
loss:  0.03929082355382657
loss:  -0.003748876283490722
loss:  0.021615619219862728
loss:  -0.024681859668979414
loss:  0.04202131673348501
loss:  0.06051693189233491
loss:  0.09311535834675053
loss:  -0.09601729572192

loss:  0.04033324007818046
loss:  -0.0035817317358576165
loss:  0.08990433592688116
loss:  0.0241428414968503
loss:  8.824084723745712e-05
loss:  0.08177573908728884
loss:  0.0013920119375300968
loss:  -0.015230873868075265
loss:  0.06630698303588123
loss:  0.10008786254348974
loss:  -0.09337514276244868
loss:  0.039868572768852126
loss:  0.059294160359344095
loss:  -0.15094877544013174
loss:  -0.021170116279951766
loss:  -0.16116035189895983
loss:  0.03353701926893305
loss:  -0.004933317296382556
loss:  -0.012665613459154734
loss:  -0.05994058121422975
loss:  -0.18345165465073168
loss:  0.017866653212449114
loss:  0.05087071327042052
loss:  -0.009367886531188717
loss:  0.03802275750280156
loss:  0.07514493376728779
loss:  -0.02126322974923881
loss:  0.12678981947559168
loss:  0.002555653254594065
loss:  0.10260325207622042
loss:  -0.03133763973025598
loss:  -0.009816946325230263
loss:  0.023841052196560936
loss:  -0.07856551132576915
loss:  -0.054919620394385915
loss:  0.0391385238454

loss:  -0.009269421089613025
loss:  0.024122983789035407
loss:  -0.07837888521129098
loss:  -0.05523544146664054
loss:  0.03888055819215941
loss:  -0.005976338811704222
loss:  0.021402359023888203
loss:  -0.025864913544053535
loss:  0.04005242142612561
loss:  0.06155136164339001
loss:  0.09307791846657504
loss:  -0.0960525050450927
loss:  0.015085754269432677
loss:  0.049729246004263816
loss:  -0.13988075478359313
loss:  -0.11020361073726127
loss:  0.0395983950658816
loss:  -0.002899333812704873
loss:  0.09055564519328947
loss:  0.02445513990758401
loss:  0.0001812908086878534
loss:  0.08126637344629926
loss:  0.0015680092682120415
loss:  -0.014299803078336615
loss:  0.06639843866780305
loss:  0.10005803965641726
loss:  -0.09414095189440819
loss:  0.04014635209931988
loss:  0.05920427909851438
loss:  -0.1508005994514227
loss:  -0.020307784691742048
loss:  -0.16174185833838636
loss:  0.032019105669510856
loss:  -0.004150732579297128
loss:  -0.012884587641486318
loss:  -0.062460840568056

loss:  -0.002386343930707436
loss:  0.09088886296750115
loss:  0.024755549951420243
loss:  0.0003485794703438419
loss:  0.08092715735879417
loss:  0.0016956109850847561
loss:  -0.013688639697699815
loss:  0.06640871512443908
loss:  0.09988773396821433
loss:  -0.09453236914784507
loss:  0.040275535412021043
loss:  0.05939351766656934
loss:  -0.1508372680348395
loss:  -0.019685942576994436
loss:  -0.16205721753567154
loss:  0.030867628144062638
loss:  -0.0035942526229804494
loss:  -0.013078785806739105
loss:  -0.06404957086268652
loss:  -0.1834167174566678
loss:  0.019167084055723236
loss:  0.052116072275718274
loss:  -0.009705305255368446
loss:  0.03609821727436767
loss:  0.07513394918515369
loss:  -0.021411711811387273
loss:  0.12722290372230474
loss:  0.002802689852661247
loss:  0.10174278541319101
loss:  -0.030622585302980457
loss:  -0.008789353997881819
loss:  0.02461080330206525
loss:  -0.0780232252650807
loss:  -0.055358798895487414
loss:  0.03869399294371825
loss:  -0.00656042241

...

loss:  0.07416062081792238
loss:  -0.021866262364948652
loss:  0.12733090573422445
loss:  0.0029485594060118373
loss:  0.10089897870412785
loss:  -0.029742182753612505
loss:  -0.00793539976111388
loss:  0.02574759776880007
loss:  -0.07709893773943216
loss:  -0.055604037753682445
loss:  0.038484338866519203
loss:  -0.0075187813095458494
loss:  0.023464499753477915
loss:  -0.029253482407950185
loss:  0.03837472882476472
loss:  0.06145797670033781
loss:  0.0924925291859574
loss:  -0.09453556737064016
loss:  0.016948113545525192
loss:  0.04894282879320621
loss:  -0.13787037386391918
loss:  -0.10929225200260752
loss:  0.04009422365550871
loss:  -0.0011132513765964003
loss:  0.0916608889656835
loss:  0.025479793041232733
loss:  0.0008359666709204447
loss:  0.08026438310874595
loss:  0.001931452404930846
loss:  -0.012218330003367023
loss:  0.06641608891601056
loss:  0.09918539035375862
loss:  -0.09516556416571471
loss:  0.04061647413711011
loss:  0.06007073943672732
loss:  -0.1511971706127212

loss:  -0.021979789411063162
loss:  0.1273360231392903
loss:  0.0029774404163389577
loss:  0.10072995637447338
loss:  -0.029501196091930682
loss:  -0.007731166675069653
loss:  0.026050132308583183
loss:  -0.07685700778674534
loss:  -0.055727879489100476
loss:  0.03845999405850401
loss:  -0.007744850535519765
loss:  0.023821842715015473
loss:  -0.02977926355760961
loss:  0.03811369510557743
loss:  0.06140347419839344
loss:  0.09233967830431436
loss:  -0.09436059368932909
loss:  0.017177083661230372
loss:  0.0488255559848729
loss:  -0.1375641941988415
loss:  -0.1091886404687774
loss:  0.04017803579497409
loss:  -0.0008089925703056813
loss:  0.0918561008251629
loss:  0.02566688782766957
loss:  0.0009629673168079986
loss:  0.08013876434011571
loss:  0.001981449231463722
loss:  -0.011866880819313594
loss:  0.06644667583811864
loss:  0.09900245746759657
loss:  -0.09527251902501888
loss:  0.04073092301282645
loss:  0.06026675678957852
loss:  -0.15131197836429933
loss:  -0.017609657166189823
l

loss:  -0.01161082271498538
loss:  0.0664830777059504
loss:  0.09888026010288842
loss:  -0.09534175854173484
loss:  0.04082801226339804
loss:  0.0604188744575692
loss:  -0.1513892432257683
loss:  -0.017321391174098186
loss:  -0.1635757347493989
loss:  0.026747713658934455
loss:  -0.0017942207771503207
loss:  -0.013979380127733768
loss:  -0.06903967732500985
loss:  -0.18386206675160321
loss:  0.020900772990279065
loss:  0.05338535256417474
loss:  -0.010167881215885355
loss:  0.03376664869003082
loss:  0.07366732479702598
loss:  -0.022057558442381567
loss:  0.12735241342790965
loss:  0.003004163436814016
loss:  0.10060448109336925
loss:  -0.029300817111733417
loss:  -0.007560291522046006
loss:  0.02630197665191021
loss:  -0.07666942156756944
loss:  -0.055870497001675574
loss:  0.03844738165516428
loss:  -0.007933765845552485
loss:  0.024122319988390834
loss:  -0.030203913559444982
loss:  0.037887287268327643
loss:  0.061372875283641395
loss:  0.0922171370619265
loss:  -0.0942452469829760

loss:  0.10052225322618039
loss:  -0.029156652779356477
loss:  -0.007433486748093561
loss:  0.026484660153388906
loss:  -0.076546207946594
loss:  -0.05600413440654781
loss:  0.038442570192940916
loss:  -0.008072829482924183
loss:  0.02434422333449831
loss:  -0.030506415523849202
loss:  0.037713743185079904
loss:  0.061365215619197326
loss:  0.09213473025425199
loss:  -0.0941820033075296
loss:  0.017517455991406763
loss:  0.04871005434225606
loss:  -0.13714000911681384
loss:  -0.10904184867904892
loss:  0.04017054814593329
loss:  -0.00037507457007345934
loss:  0.09215340297575092
loss:  0.025977267069779684
loss:  0.001144656916443526
loss:  0.07997396485624551
loss:  0.0020586997520076764
loss:  -0.011353814773850371
loss:  0.06653405051555371
loss:  0.09877354275798632
loss:  -0.09540341378798721
loss:  0.040939223428789445
loss:  0.06058100723887317
loss:  -0.15145398202792346
loss:  -0.01704626410305489
loss:  -0.16387913363253506
loss:  0.026262168464570794
loss:  -0.00159060877502

loss:  -0.18421115114910103
loss:  0.021254681741079574
loss:  0.05355160557317837
loss:  -0.01029882248917818
loss:  0.033403350142973215
loss:  0.07342509101536504
loss:  -0.022125265159456856
loss:  0.12742044253096038
loss:  0.0030518371363286415
loss:  0.1004493167544126
loss:  -0.02901575448345759
loss:  -0.00730358045259655
loss:  0.02666598864491441
loss:  -0.07643925695857325
loss:  -0.056169253596764404
loss:  0.0384417773998015
loss:  -0.008212964901298844
loss:  0.024569575180718284
loss:  -0.03080299650847309
loss:  0.03753010998135113
loss:  0.061375275132169636
loss:  0.0920614981897274
loss:  -0.0941398560676813
loss:  0.01767728138940095
loss:  0.048688913594859294
loss:  -0.1369668567696477
loss:  -0.10897630631547364
loss:  0.04009944674140831
loss:  -0.00019614430944481578
loss:  0.0922849890962853
loss:  0.02613152965687176
loss:  0.0012187139103451775
loss:  0.07990808926238825
loss:  0.002094137675020342
loss:  -0.011133356588311268
loss:  0.06659112934813272
los

In [21]:
model.layers[0].weights

array([[0.62341708, 0.22141651, 1.01651053, 0.18281256, 0.56587321,
        0.19998137, 0.13165999, 0.49164701, 1.00874571, 0.6848976 ,
        0.3274529 , 0.85553652, 1.02051612, 0.34685166, 0.70100048],
       [0.42166862, 0.45898984, 0.97993727, 0.12555674, 0.14270847,
        0.81963009, 0.26589832, 0.37199716, 0.16039767, 0.46331265,
        0.8975801 , 0.46642247, 0.82809756, 0.70296888, 0.58031047],
       [0.04862323, 0.68624527, 0.0478799 , 0.97834832, 0.98241865,
        0.87888137, 0.55949521, 0.72542193, 0.83957885, 0.9087448 ,
        0.23631082, 0.57385335, 0.34381308, 0.44530451, 0.4570737 ],
       [0.08419168, 0.89449466, 0.31362677, 0.54537152, 0.70038043,
        0.6199248 , 0.41146448, 0.82648106, 0.56744998, 0.81073206,
        0.34764677, 0.90247799, 0.42088705, 0.45565857, 0.92168759],
       [0.95976192, 0.68829568, 0.0875807 , 0.20438143, 0.38398253,
        1.04142386, 0.89678986, 0.30935905, 0.33415588, 0.47169037,
        0.41582344, 0.84138683, 0.90752291, 

In [22]:
model.layers[1].weights

array([[1.07565032],
       [0.48520289],
       [0.95297706],
       [0.86956099],
       [0.74868515],
       [0.62611971],
       [0.56362797],
       [0.96913002],
       [0.78034542],
       [0.27390438],
       [0.79469716],
       [0.19330302],
       [0.43978274],
       [0.18076378],
       [1.00111817]])

# View Results on Test Set

In [23]:
test_predictions = model.predict(x_test)
test_predictions = np.array([1 if x>0.5 else 0 for x in test_predictions], dtype=np.uint8)

In [24]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, test_predictions)

array([[115,   6],
       [  2,  65]], dtype=int64)

In [25]:
from sklearn.metrics import accuracy_score

print("accuracy: ", accuracy_score(y_test, test_predictions))

accuracy:  0.9574468085106383
